In [ ]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - Client Side Processing

## Data Fusion


In this interactive notebook we will show some usage examples of the Client Side Processing functionality of the openEO Python client, interacting with STAC Catalogs from different providers.

## Requirements

To use this functionality, you need `Python>=3.9`.

You can install it using:
    `pip install openeo[localprocessing]`

<div class="alert alert-block alert-warning">
This functionality is still under development and the installation procedure might change.
Please refer to official documentation page for the most up to date instructions:
    
<a href="url">https://open-eo.github.io/openeo-python-client/cookbook/localprocessing.html</a>

## Sentinel-2 and Landsat-8 data

We are going to work with the Landsat-8 data provided by the Microsoft Planetary Computer and Sentinel-2 data provided by the element84.

We load only the data of our area of interest, selecting the red and near infrared bands for a limited period of time.
We also set the conditions on the cloud coverage, which must be less than 50%.

### Load the Sentinel-2 data

In [ ]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2023-06-01", "2023-07-30"]
bands = ["red","nir"]

properties = {"eo:cloud_cover": dict(lt=50)}
s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands,
   properties=properties,
)

s2_cube.execute()

### Load the Landsat-8 data

In [ ]:
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2"
temporal_extent = ["2023-06-01", "2023-07-30"]
spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
bands = ["red","nir08"]
properties = {"eo:cloud_cover": dict(lt=50)}
l8_cube = local_conn.load_stac(url=url,
                    spatial_extent=spatial_extent,
                    temporal_extent=temporal_extent,
                    bands=bands,
                    properties=properties)
l8_cube.execute()

### Resample Sentinel-2

Sentinel-2 and Landsat-8 have different resolutions. We use this process to resample and align Sentinel-2 to Landsat-8, from 10 meters to 30 meters.

In [ ]:
s2_cube_30m = s2_cube.resample_cube_spatial(target=l8_cube,method="average")

### Compute the NDVI

We compute the Normalized Difference Vegetation Index with data from both satellite missions.

In [ ]:
s2_red = s2_cube_30m.band("red")
s2_nir = s2_cube_30m.band("nir")
s2_ndvi = (s2_nir - s2_red) / (s2_nir + s2_red)
s2_ndvi = s2_ndvi.add_dimension(name="bands",type="bands",label="s2_ndvi")

In [ ]:
l8_red = l8_cube.band("red")
l8_nir = l8_cube.band("nir08")
l8_ndvi = (l8_nir - l8_red) / (l8_nir + l8_red)
l8_ndvi = l8_ndvi.add_dimension(name="bands",type="bands",label="l8_ndvi")

### Aggregate over time
Since the acquisitions of the different satellites do not happen at the same time, we aggregate the data for each month and later compare the monthly median of the NDVI.

In [ ]:
s2_ndvi_agg = s2_ndvi.aggregate_temporal_period(period="month",reducer="median")
l8_ndvi_agg = l8_ndvi.aggregate_temporal_period(period="month",reducer="median")

### Merge into a single datacube

We finally merge all the previous steps in a single datacube for convenience. Check out the visualization of the openEO process graph we just created:

In [ ]:
s2_l8_ndvi = s2_ndvi_agg.merge_cubes(l8_ndvi_agg)
s2_l8_ndvi

### Compute the result

In [ ]:
s2_l8_ndvi_xr = s2_l8_ndvi.execute()
s2_l8_ndvi_xr

### Visualize the monthly median NDVI values:

In [ ]:
s2_l8_ndvi_xr[0,1].plot.imshow(vmin=0,vmax=1)

In [ ]:
s2_l8_ndvi_xr[1,1].plot.imshow(vmin=0,vmax=1)